<h1>Model Training</h1>

In this notebook, we will use the Amazon SageMaker open source XGBoost container (https://github.com/aws/sagemaker-xgboost-container) to train a simple binary classification model, using the pre-processed data generated in the previous step by the processing job.
Using XGBoost as a framework provides more flexibility than using it as a built-in algorithm as it enables more advanced scenarios that allow pre-processing and post-processing scripts or any kind of custom logic to be incorporated into your training script.

Let's define the variables first.

In [ ]:
# Check SageMaker Python SDK version
import sagemaker
print(sagemaker.__version__)

def versiontuple(v):
    return tuple(map(int, (v.split("."))))

if versiontuple(sagemaker.__version__) < versiontuple('2.22.0'):
    raise Exception("This notebook requires at least SageMaker Python SDK version 2.22.0. Please install it via pip.")

In [ ]:
import boto3
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
prefix = 'endtoendmlsm'

print(region)
print(role)
print(bucket_name)

In [ ]:
%store -r experiment_name
%store -r trial_name

print(experiment_name)
print(trial_name)

## Training

The training code is implemented in the `source_dir/training.py` file.

The script parses arguments that are passed when the XGBoost Docker container code invokes the script for execution. These arguments represent the hyperparameters that you specify when strarting the training job plus the location of training and validation data. Then, we load training and validation data and execute XGBoost training with the provided parameters.

<strong>Note</strong>: this behavior, named Script Mode execution, is enabled by a library that is installed in the XGBoost container (sagemaker-training-toolkit, https://github.com/aws/sagemaker-training-toolkit) and facilitates the development of SageMaker-compatible Docker containers.

In [ ]:
!pygmentize source_dir/training.py

Once we have our script ready, we can leverage on the XGBoost estimator of the Amazon SageMaker Python SDK to start training.

In [ ]:
from sagemaker.xgboost import XGBoost

hyperparameters = {
    "max_depth": "3",
    "eta": "0.1",
    "gamma": "6",
    "min_child_weight": "6",
    "silent": "0",
    "objective": "reg:logistic",
    "num_round": "20"
}

entry_point='training.py'
source_dir='source_dir/'
output_path = 's3://{0}/{1}/output/'.format(bucket_name, prefix)
code_location = 's3://{0}/{1}/code'.format(bucket_name, prefix)

estimator = XGBoost(
    base_job_name="end-to-end-ml-sm-xgb",
    entry_point=entry_point,
    source_dir=source_dir,
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    framework_version="0.90-2",
    py_version="py3",
    role=role
)

In [ ]:
# Experiment tracking configuration
experiment_config={
    "ExperimentName": experiment_name,
    "TrialName": trial_name,
    "TrialComponentDisplayName": "xgboost-training",
}

train_config = sagemaker.TrainingInput('s3://{0}/{1}/data/preprocessed/train/'.format(
    bucket_name, prefix), content_type='text/csv')
val_config = sagemaker.TrainingInput('s3://{0}/{1}/data/preprocessed/val/'.format(
    bucket_name, prefix), content_type='text/csv')

estimator.fit({'train': train_config, 'validation': val_config},
              experiment_config=experiment_config)

### Experiment analytics

You can visualize experiment analytics either from Amazon SageMaker Studio Experiments plug-in or using the SDK from a notebook, as follows:

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

analytics = ExperimentAnalytics(experiment_name=experiment_name)
analytics.dataframe()

After the training is completed, the serialized model will be saved in the S3 `output_location` defined above.
You can now move to the next notebook in the **04_deploy_model** folder to see how to use that model for inference.